# import packages

In [1]:
import tweepy
import yaml
import json
import sqlite3
from datetime import datetime
import pandas as pd

# database connection

In [2]:
# establish database connection
conn = sqlite3.connect('tweet_example.db')
# get the cursor object
cur  = conn.cursor()

# data exploration: top 10 most followed user

In [6]:
query = """ SELECT user_id, followers_count FROM user_info ORDER BY followers_count DESC;"""
cur.execute(query)
result = cur.fetchall()
columns = [description[0] for description in cur.description]
result_df = pd.DataFrame(result, columns=columns)
result_df

,user_id,followers_count
0,27649623,541388
1,14074515,346310
2,3619536852,24799
3,104854735,20093
4,450203723,3652
5,2863839994,3601
6,533604777,1294
7,1181029367578845184,1284
8,3095182126,1267
9,73229753,1170


In [7]:
query = """SELECT hashtags FROM tweet_info;"""
cur.execute(query)
result = cur.fetchall()
columns = [description[0] for description in cur.description]
result_df = pd.DataFrame(result, columns=columns)
result_df

,hashtags
0,Ida
1,
2,
3,
4,
5,ida
6,
7,
8,Ida
9,


In [8]:
query = """SELECT tweet_text FROM tweet_info;"""
cur.execute(query)
result = cur.fetchall()
columns = [description[0] for description in cur.description]
result_df = pd.DataFrame(result, columns=columns)
result_df

,tweet_text
0,"As of today, Orleans Parish is 98% restored af..."
1,RT @Flamengo: Amanhã tem Mengão na semifinal d...
2,Could use a little extra after Hurricane Ida A...
3,"Post-Ida, lots of patients required special me..."
4,Jævla bra sagt @ErikFollestad
5,RT @joshnsanchez: People in Louisiana scooping...
6,RT @tuitamad: o bil vai entrar na fazenda pra ...
7,I'm raising money for Hurricane IDA. Click to ...
8,RT @femaregion2: If you suffered losses from #...
9,"PREMIO! Te has ganado boleto de ida al paro, p..."


# sentiment analyzer

In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
combined_pat = r'|'.join((r'@[A-Za-z0-9_]+', r'RT\s@\S+', \
                          r'https?://[^ ]+',r'www.[^ ]+'))
analyser=SentimentIntensityAnalyzer()
def get_tweet_sentiment(tweet_text):
    
    filtered_text = re.sub(combined_pat, '', tweet_text)
    vader_score=analyser.polarity_scores(filtered_text)
    
    if (vader_score['compound'] <= -0.05):
        sentiment=-1
    elif (vader_score['compound'] >=0.05):
        sentiment=1
    else:
        sentiment=0
        
    return sentiment

# iterate through the tweet_texts

In [13]:
pos = 0
neg = 0
neutral = 0
for index, row in result_df.iterrows():
    tweet_text = row['tweet_text']
    sentiment = get_tweet_sentiment(tweet_text)
    if sentiment == -1:
        neg += 1
    elif sentiment == 1:
        pos += 1
    else:
        neutral += 1
print(f"pos: {pos}, neg: {neg}, neutral: {neutral}.")
    

pos: 8, neg: 6, neutral: 5.
